In [2]:
import pandas as pd
import os

In [3]:
os.getcwd()

'/Users/markgiannini/Desktop/7PMark/Sunny'

In [ ]:
#Credentials
#username: p-tbernardin@apptio-poc.com

In [4]:
#Read in 3 data sets

cloud_invoice = pd.read_csv('./Data/Cloud_Invoice.txt', sep = '\t')
cloud_invoice_2019_09_10 = pd.read_csv('./Data/Cloud_Invoice_2019-09-10.txt', sep = '\t')
cloud_invoice_2019_09_13 = pd.read_csv('./Data/Cloud_Invoice_2019-09-13.txt', sep = '\t')



#### Cloud Invoice

In [104]:
cloud_invoice.shape

(2373824, 15)

In [88]:
#import qgrid#
#widget = qgrid.show_grid(cloud_invoice)
#widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [133]:
total_cnt = len(cloud_invoice)
d = cloud_invoice.isnull().sum(axis = 0)
d_pd = pd.DataFrame(d)
d_pd.columns = ['null_cnt']
d_pd
d_pd['total_rows'] = total_cnt
d_pd['null_prct'] = d_pd.null_cnt/d_pd.total_rows
d_pd.sort_values(by = ['null_prct'])


,null_cnt
__row,0
__tenant,0
__date,0
Vendor,0
Seller,0
ProductName,0
ServiceName,0
TransactionType,0
LeaseType,0
Operation,61


In [132]:
cloud_invoice.shape

#Raw Data Shape: (41624032, 22)

(2373824, 15)

In [107]:
s3_samplerows= cloud_invoice.head(5)
s3_samplerows.to_csv('s3_samplerows.csv')

#### Column Differences

In [ ]:

#SHARED COLS: __row, __tenant, __date, Vender, Seller, ProductName, ServiceName, TransactionType
            #Lease Type, Operation, UsageQuantityUnit

#IN RAW, not AGG: __batch, Organization[Name, Size, Sector, ID], Item Description, UsageQuantity 

#IN AGG, not RAW: __underlyingRowCount, _+Cost, _-Cost, UsageQuantityUnit, __errors


##### Examining __underlyingRowCount column

In [16]:
cond1 = (cloud_invoice.__date == 'September 2015')
cond2 = (cloud_invoice.__tenant == 'Tenant B')
cond3 = (cloud_invoice.ProductName == 'Amazon CloudFront')
cond4 = (cloud_invoice.Operation == 'PUT')

test_case = cloud_invoice[cond1 & cond2 & cond3 & cond4]
test_case.sort_values(['__date', 'Vendor', 'Seller', 'ProductName', 'ServiceName'])

,__row,__tenant,__date,Vendor,Seller,ProductName,ServiceName,TransactionType,LeaseType,Operation,UsageQuantityUnit,_+Cost,_-Cost,__underlyingRowCount,__errors
759,759,Tenant B,September 2015,Amazon,Amazon,Amazon CloudFront,AWS CloudFront,Usage,On-Demand,PUT,BytesTransferred,0.0,NaN,4,0
7788,7788,Tenant B,September 2015,Amazon,Amazon,Amazon CloudFront,AWS CloudFront,Usage,On-Demand,PUT,Requests,0.0,NaN,2,0


In [ ]:
cond1 = (cloud_invoice.__date == 'September 2015')
cond2 = (cloud_invoice.__tenant == 'Tenant B')
cond3 = (cloud_invoice.ProductName == 'Amazon CloudFront')
cond4 = (cloud_invoice.Operation == 'PUT')

test_case = cloud_invoice[cond1 & cond2 & cond3 & cond4]
test_case.sort_values(['__date', 'Vendor', 'Seller', 'ProductName', 'ServiceName'])

In [20]:
#Getting Total Count of Underlying Rows to See % of Total Rows Here

raw_data_rows = 41624032

underlying_row_cnt = sum(cloud_invoice.__underlyingRowCount)

print('There are {} ({}) underlying rows in Agg. Data vs {} in raw set'.format(underlying_row_cnt, underlying_row_cnt/raw_data_rows, raw_data_rows))

There are 16344731 (0.3926753419755203) underlying rows in Agg. Data vs 41624032 in raw set


In [110]:
#Time Period:

date = cloud_invoice.__date

min(date), max(date), len(date.unique()) #Same Values in Raw

#Manual Inspection showed both range: July 2015 - September 2019

cloud_invoice.columns

Index(['__row', '__tenant', '__date', 'Vendor', 'Seller', 'ProductName',
       'ServiceName', 'TransactionType', 'LeaseType', 'Operation',
       'UsageQuantityUnit', '_+Cost', '_-Cost', '__underlyingRowCount',
       '__errors'],
      dtype='object')

In [128]:
cloud_invoice.loc[:,['_+Cost', '_-Cost']].describe()

,_+Cost,_-Cost
count,2.198141e+06,1.896130e+05
mean,2.474250e+03,-4.623883e+03
std,6.374935e+04,1.580027e+05
min,0.000000e+00,-6.148790e+07
25%,0.000000e+00,-1.993800e+02
50%,2.150000e-02,-1.124000e+01
75%,2.418670e+01,-2.921000e-01
max,6.451399e+07,-0.000000e+00


In [131]:
pd.options.display.float_format = '{:.2f}'.format
cloud_invoice.describe(include = 'all')

,__row,__tenant,__date,Vendor,Seller,ProductName,ServiceName,TransactionType,LeaseType,Operation,UsageQuantityUnit,_+Cost,_-Cost,__underlyingRowCount,__errors
count,2373824.00,2373824,2373824,2373824,2373824,2373824,2373824,2373824,2373824,2373763,2373824,2198141.00,189613.00,2373824.00,2373824.00
unique,nan,220,51,3,23,1109,1012,6,4,2276,56,nan,nan,nan,nan
top,nan,Tenant BEG,July 2019,Amazon,Amazon,Amazon Simple Storage Service,AWS S3,Usage,On-Demand,(not set),Requests,nan,nan,nan,nan
freq,nan,36626,103446,2165604,2144418,650000,658770,2060885,2319727,126903,584602,nan,nan,nan,nan
mean,1186911.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2474.25,-4623.88,6.89,0.00
std,685264.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63749.35,158002.65,78.49,0.00
min,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,-61487901.00,1.00,0.00
25%,593455.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,-199.38,1.00,0.00
50%,1186911.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,-11.24,2.00,0.00
75%,1780367.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.19,-0.29,5.00,0.00


In [125]:
len(cloud_invoice.Operation.unique())

2277

In [27]:
#Number of Tenants

tenants = pd r.__tenant

len(tenants.unique())

#1,273 tenants in raw data

220

In [158]:
#Number of Tenants Over Time

panel_over_time = pd.DataFrame(cloud_invoice.groupby(by = ['__date'])['__tenant'].nunique()).reset_index()

panel_over_time.__date = pd.to_datetime(panel_over_time.__date)

panel_over_time.sort_values(by = ['__date'])

,__date,__tenant
21,2015-07-01,1
4,2015-08-01,1
46,2015-09-01,2
42,2015-10-01,2
38,2015-11-01,7
9,2015-12-01,15
17,2016-01-01,22
13,2016-02-01,44
30,2016-03-01,52
0,2016-04-01,61


In [33]:
#Number of Vendors
vendors = cloud_invoice.Vendor

len(vendors.unique()), vendors.unique()


(3, array(['Amazon', 'Azure', 'GCP'], dtype=object))

In [35]:
#Number of Sellers

sellers = cloud_invoice.Seller

len(sellers.unique()), sellers.unique()

#Raw data has 39 sellers


(23, array(['Amazon', 'Azure', 'Google', 'AWS Marketplace', 'Cognosys Inc.',
        'Azure Marketplace', 'Palo Alto Networks', 'Bitnami', 'SendGrid',
        'F5 Networks', 'Aqua Security Software Inc.', 'DataStax',
        'RStudio', 'Google Click to Deploy', 'Canonical', 'Vertica',
        '#n31781 #n4347', 'OpenVPN Inc.', 'Genymobile', 'Neo0j', 'Actifio',
        'Jetware', 'Grakn Labs'], dtype=object))

In [54]:
#Types of Transactions
 
transaction_types = cloud_invoice.TransactionType
len(transaction_types.unique()), transaction_types.unique()

#Raw data has same types of charges 

(6, array(['Usage', 'Recurring', 'Credit', 'Tax', 'Rounding', 'One-time'],
       dtype=object))

In [65]:
#Avg Number of Underlying Rows for Each Transaction Type
cloud_invoice.loc[:, ['TransactionType', '__underlyingRowCount']].groupby(by = 'TransactionType').mean() \

,__underlyingRowCount
TransactionType,
Credit,4.152639
One-time,5.735681
Recurring,9.715448
Rounding,1.000000
Tax,6.633247
Usage,7.084822


In [66]:
cloud_invoice[cloud_invoice.TransactionType == 'Rounding']

,__row,__tenant,__date,Vendor,Seller,ProductName,ServiceName,TransactionType,LeaseType,Operation,UsageQuantityUnit,_+Cost,_-Cost,__underlyingRowCount,__errors
210,210,Tenant B,January 2018,Amazon,Amazon,Rounding,AWS Administrative,Rounding,On-Demand,(not set),(not set),0.5456,NaN,1,0
399,399,Tenant B,December 2015,Amazon,Amazon,Rounding,AWS Administrative,Rounding,On-Demand,(not set),(not set),0.3649,NaN,1,0
519,519,Tenant B,November 2016,Amazon,Amazon,Rounding,AWS Administrative,Rounding,On-Demand,(not set),(not set),0.4436,NaN,1,0
593,593,Tenant B,July 2018,Amazon,Amazon,Rounding,AWS Administrative,Rounding,On-Demand,(not set),(not set),0.2865,NaN,1,0
1066,1066,Tenant B,March 2016,Amazon,Amazon,Rounding,AWS Administrative,Rounding,On-Demand,(not set),(not set),0.3504,NaN,1,0
1143,1143,Tenant B,May 2018,Amazon,Amazon,Rounding,AWS Administrative,Rounding,On-Demand,(not set),(not set),0.5174,NaN,1,0
1986,1986,Tenant B,September 2015,Amazon,Amazon,Rounding,AWS Administrative,Rounding,On-Demand,(not set),(not set),0.3324,NaN,1,0
2240,2240,Tenant B,April 2019,Amazon,Amazon,Rounding,AWS Administrative,Rounding,On-Demand,(not set),(not set),0.0034,NaN,1,0
2430,2430,Tenant B,August 2018,Amazon,Amazon,Rounding,AWS Administrative,Rounding,On-Demand,(not set),(not set),0.5539,NaN,1,0
2649,2649,Tenant B,April 2018,Amazon,Amazon,Rounding,AWS Administrative,Rounding,On-Demand,(not set),(not set),0.5696,NaN,1,0


In [71]:
#Seeing if there is just one record per tenant per month
df = cloud_invoice[cloud_invoice.TransactionType == 'Rounding'].groupby(by = ['__tenant', '__date', 'TransactionType']) \
    .mean()[['__underlyingRowCount']].reset_index()
df[df.__underlyingRowCount != 1]

#empty table -- rounding only occurs once per tenant per month

,__tenant,__date,TransactionType,__underlyingRowCount


In [79]:
#seeing if each charge only occurs once per month per tenant
df = cloud_invoice.drop(['__row', 'Operation', '__underlyingRowCount', '__errors'], axis = 1)
sum(df.duplicated(keep = False))

681640

In [75]:
df.drop_duplicates

<bound method DataFrame.drop_duplicates of            __tenant          __date  Vendor           Seller  \
0          Tenant B     August 2018  Amazon           Amazon   
1          Tenant B   February 2017  Amazon           Amazon   
2          Tenant B        May 2017  Amazon           Amazon   
3          Tenant B  September 2019  Amazon           Amazon   
4          Tenant B      March 2019  Amazon           Amazon   
5          Tenant B    January 2019  Amazon           Amazon   
6          Tenant B      March 2018  Amazon           Amazon   
7          Tenant B     August 2017  Amazon           Amazon   
8          Tenant B   February 2017  Amazon           Amazon   
9          Tenant B   February 2016  Amazon           Amazon   
10         Tenant B   December 2016  Amazon           Amazon   
11         Tenant B       June 2018  Amazon           Amazon   
12         Tenant B    October 2018  Amazon           Amazon   
13         Tenant B        May 2016  Amazon           Amazon 

#### Transaction Type

In [82]:
#Credit

cond1 = (cloud_invoice.__date == 'April 2019')
cond2 = (cloud_invoice.__tenant == 'Tenant EL')
cond3 = (cloud_invoice.ProductName == 'Amazon Simple Storage Service')
cond4 = (cloud_invoice.Operation == 'S0-SIATransition')

test_case = cloud_invoice[cond1 & cond2 & cond3 & cond4]
test_case.sort_values(['__date', 'Vendor', 'Seller', 'ProductName', 'ServiceName'])

#credits appear to be deductions on service - paired with a usage line? 

,__row,__tenant,__date,Vendor,Seller,ProductName,ServiceName,TransactionType,LeaseType,Operation,UsageQuantityUnit,_+Cost,_-Cost,__underlyingRowCount,__errors
489529,489529,Tenant EL,April 2019,Amazon,Amazon,Amazon Simple Storage Service,AWS S3,Credit,On-Demand,S0-SIATransition,Requests,NaN,-2.164,2,0
492095,492095,Tenant EL,April 2019,Amazon,Amazon,Amazon Simple Storage Service,AWS S3,Usage,On-Demand,S0-SIATransition,Requests,18.0333,NaN,2,0


In [166]:
cloud_invoice[(cloud_invoice['_+Cost'] > 1000000) & (cloud_invoice.TransactionType != 'One-time')]

,__row,__tenant,__date,Vendor,Seller,ProductName,ServiceName,TransactionType,LeaseType,Operation,UsageQuantityUnit,_+Cost,_-Cost,__underlyingRowCount,__errors
68398,68398,Tenant K,May 2018,Amazon,Amazon,Amazon Relational Database Service,AWS RDS,Usage,On-Demand,Not Applicable,Unknown,1665446.61,nan,3,0
68483,68483,Tenant K,June 2018,Amazon,Amazon,Amazon Relational Database Service,AWS RDS,Usage,On-Demand,Not Applicable,Unknown,1562624.59,nan,2,0
78907,78907,Tenant K,April 2018,Amazon,Amazon,Amazon Relational Database Service,AWS RDS,Usage,On-Demand,Not Applicable,Unknown,1646786.79,nan,3,0
80543,80543,Tenant K,June 2017,Azure,Azure,Microsoft.Compute,Azure Compute,Usage,On-Demand,Compute Hours,Hours,1009748.04,nan,235,0
86021,86021,Tenant K,March 2018,Amazon,Amazon,Amazon Relational Database Service,AWS RDS,Usage,On-Demand,Not Applicable,Unknown,1393720.52,nan,2,0
98932,98932,Tenant L,April 2018,Amazon,Amazon,Amazon Elastic Compute Cloud,AWSEC2,Usage,Reserved,RunInstances,UsageHours,1116405.65,nan,173,0
99834,99834,Tenant L,October 2017,Amazon,Amazon,Amazon Elastic Compute Cloud,AWSEC2,Usage,On-Demand,RunInstances,UsageHours,1475993.17,nan,198,0
100853,100853,Tenant L,April 2018,Amazon,Amazon,Amazon Elastic Compute Cloud,AWSEC2,Usage,On-Demand,RunInstances,UsageHours,2333679.96,nan,241,0
101151,101151,Tenant L,April 2019,Amazon,Amazon,Amazon Elastic Compute Cloud,AWSEC2,Usage,On-Demand,RunInstances,UsageHours,1528344.47,nan,336,0
102167,102167,Tenant L,September 2018,Amazon,Amazon,Amazon Elastic Compute Cloud,AWSEC2,Usage,On-Demand,RunInstances,UsageHours,2317550.34,nan,251,0


In [1]:
#Various Groupings of Vendor/Seller/Product Below

In [8]:
cloud_invoice.loc[:, ['Vendor', 'Seller', 'ProductName']].drop_duplicates() #.groupby(by = ['Vendor', 'Seller', 'ProductName'], axis = 1)


,Vendor,Seller,ProductName
0,Amazon,Amazon,Amazon Simple Storage Service
1,Amazon,Amazon,Amazon Route 53
2,Amazon,Amazon,AWS Key Management Service
3,Amazon,Amazon,AWS Cost Explorer
6,Amazon,Amazon,Amazon Simple Queue Service
8,Amazon,Amazon,Amazon Simple Notification Service
9,Amazon,Amazon,Amazon Elastic Compute Cloud
10,Amazon,Amazon,Amazon RDS Service
16,Amazon,Amazon,AmazonCloudWatch
18,Amazon,Amazon,Amazon SimpleDB


#### Notes on Cloud Invoice


In [9]:
cloud_invoice.loc[:,['Vendor', 'Seller', 'ProductName', '_+Cost']].groupby(by = ['Vendor', 'Seller', 'ProductName']).sum().sort_values('_+Cost', ascending = False)

_+Cost
Vendor Seller            ProductName                                                     
Amazon Amazon            Amazon Elastic Compute Cloud                        2.817861e+09
GCP    Google            GCP Compute Engine                                  5.126769e+08
Amazon Amazon            Amazon Relational Database Service                  4.108761e+08
                         Amazon Simple Storage Service                       3.264472e+08
                         AWS Premium Support                                 1.362486e+08
Azure  Azure             Microsoft.Compute                                   8.796196e+07
Amazon Amazon            Amazon Redshift                                     8.485982e+07
                         Amazon DynamoDB                                     8.019632e+07
                         Amazon RDS Service                                  7.863150e+07
                         Amazon ElastiCache                                  7.437468e+07
                         Amazon Elasticsearch Service                        7.088840e+07
                         AmazonCloudWatch                                    6.191660e+07
GCP    Google            GCP Cloud Storage                                   5.508171e+07
                         GCP Cloud Dataflow                                  5.110496e+07
                         GCP BigQuery                                        3.583264e+07
                         GCP Cloud Bigtable                                  2.839496e+07
Amazon Amazon            Amazon Elastic MapReduce                            2.834894e+07
                         Amazon WorkSpaces                                   2.553870e+07
                         Amazon CloudFront                                   2.445532e+07
                         Amazon Elastic File System                          2.443331e+07
                         AWS Direct Connect                                  2.301637e+07
                         AWS CloudTrail                                      1.866692e+07
Azure  Azure             Microsoft.Capacity                                  1.864997e+07
Amazon Amazon            Amazon Simple Queue Service                         1.843405e+07
                         AWS Support (Business)                              1.800643e+07
                         VAT                                                 1.589655e+07
GCP    Google            GCP Cloud Pub/Sub                                   1.566519e+07
Azure  Azure             Microsoft.Storage                                   1.480436e+07
                         Microsoft.Sql                                       1.467951e+07
Amazon Amazon            Amazon Kinesis                                      1.353450e+07
...                                                                                   ...
       AWS Marketplace   G-vTAP Controller                                   0.000000e+00
                         Informatica Remote Access Jump Server               0.000000e+00
                         Informatica Hadoop-Cluster Image for EDC            0.000000e+00
Azure  Azure Marketplace Barracuda CloudGen WAF for Azure Free 00-day        0.000000e+00
                         Barracuda CloudGen WAF for Azure-Barracuda Clou...  0.000000e+00
Amazon AWS Marketplace   Hyperglance - up to 000 Nodes                       0.000000e+00
                         Hyperglance - up to 0,000 Nodes                     0.000000e+00
                         Hortonworks Data Cloud - HDP Services               0.000000e+00
                         Hortonworks Data Cloud - Controller Service         0.000000e+00
Azure  Azure Marketplace Bing Maps API for Enterprise-Internal Website T...  0.000000e+00
Amazon AWS Marketplace   Graylog Powered by Intuz                            0.000000e+00
                         Git                                                 0.000000e+00
                         GigaVUE 

In [10]:
cloud_invoice.loc[:,['Vendor', 'Seller', '_+Cost']].groupby(by = ['Vendor', 'Seller']).sum().sort_values('_+Cost', ascending = False  )

_+Cost
Vendor Seller                                   
Amazon Amazon                       4.486764e+09
GCP    Google                       7.203527e+08
Azure  Azure                        1.954566e+08
Amazon AWS Marketplace              3.566854e+07
Azure  Azure Marketplace            2.366947e+05
GCP    F5 Networks                  1.623858e+05
       Palo Alto Networks           1.051563e+05
       Aqua Security Software Inc.  2.809082e+03
       RStudio                      9.748161e+02
       SendGrid                     1.396974e+02
       Grakn Labs                   2.390280e+01
       Genymobile                   1.303400e+00
       Cognosys Inc.                1.880000e-02
       Jetware                      0.000000e+00
       OpenVPN Inc.                 0.000000e+00
       Neo0j                        0.000000e+00
       Google Click to Deploy       0.000000e+00
       Bitnami                      0.000000e+00
       #n31781 #n4347               0.000000e+00
       Actifio                      0.000000e+00
       DataStax                     0.000000e+00
       Canonical                    0.000000e+00
       Vertica                      0.000000e+00

In [11]:
cloud_invoice.loc[:,['Vendor', '_+Cost']].groupby(by = ['Vendor']).sum().sort_values('_+Cost', ascending = False  )

,_+Cost
Vendor,
Amazon,4.522432e+09
GCP,7.206242e+08
Azure,1.956933e+08


In [12]:
# Lots of venders, seems to be order alphabetically and then just adding on letters/looping around past 26
# Vendors seem to be repeated often
#     Apparent Tree Flow: Vendor --> Seller --> Product/Service Name 



In [13]:
cloud_invoice.groupby('Vendor').count()

,__row,__tenant,__date,Seller,ProductName,ServiceName,TransactionType,LeaseType,Operation,UsageQuantityUnit,_+Cost,_-Cost,__underlyingRowCount,__errors
Vendor,,,,,,,,,,,,,,
Amazon,2165604,2165604,2165604,2165604,2165604,2165604,2165604,2165604,2165543,2165604,1999170,177045,2165604,2165604
Azure,153269,153269,153269,153269,153269,153269,153269,153269,153269,153269,153253,16,153269,153269
GCP,54951,54951,54951,54951,54951,54951,54951,54951,54951,54951,45718,12552,54951,54951
